# ICRU 2021 Data Analysis


In [261]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import seaborn as sns

## File Import
Req. Libraries
- xlrd 2.0.1 (Library for developers to extract data from Microsoft Excel (tm) .xls spreadsheet files)
- openpyxl 3.0.9 (A Python library to read/write Excel 2010 xlsx/xlsm files)

File Name
- icru_2021_data_sheet0 : (원본)ICRU데이터_rev1_220108_테스트.xlsx / Sheet0
- icru_2021_data_sheet1 : (원본)ICRU데이터_rev1_220108_테스트.xlsx / Sheet1

In [249]:
icru_2021_data_sheet0 = pd.read_excel('(원본)ICRU데이터_rev1_220108_테스트.xlsx',sheet_name=0,index_col=False)
icru_2021_data_sheet1 = pd.read_excel('(원본)ICRU데이터_rev1_220108_테스트.xlsx',sheet_name=1,index_col=False)
# icru_2021_data_sheet0.info()
# icru_2021_data_sheet1.info()

## Cleansing

sheet0 [2162 rows x 16 columns]
- id(int)
- type(category)
- gender(Binary) : 1(남성), 0(여성)
- year(int)
- work1-3(category)
- expF1-9(category)

sheet1 [2160 rows x 38 columns]
- id(int)
- type(category)
- F1-9_R, F1-9_E, F1-9_A, F1-9_D (int)


In [250]:
'''
A. icru_2021_data_sheet0
_column index
    id(int)
    type(category)
    gender(Binary) : 1(남성), 0(여성)
    year(int)
    work1-3(category) 
    expF1-9(category)
'''
sheet0 = icru_2021_data_sheet0.copy()
### Delete '테스트' data
col = sheet0.columns[1]
for key in ['테스트','d']:
    testRow = sheet0[sheet0[col]==key].index
    sheet0.drop(testRow,inplace=True)

### Column Clean-UP
sheet0.drop(sheet0.columns[16:], axis=1, inplace=True)
sheet0.drop(sheet0.columns[1], axis=1, inplace=True)
orgColumn = list(sheet0.columns)
renamedColumn = ['id', 'gender', 'year',
                'work1', 'work2', 'work3',
                'expF1', 'expF2', 'expF3', 'expF4', 'expF5', 'expF6', 'expF7', 'expF8', 'expF9']
sheet0.columns = renamedColumn
columnRef0 = np.array([renamedColumn,orgColumn]).transpose()

### Row Clean-UP
sheet0['gender'] = sheet0['gender'].map({'남':1,'여':0}).astype('boolean')
sheet0['year'] = sheet0['year'].astype('int64')



In [251]:
'''
B. icru_2021_data_sheet1
_column index
    id(int)
    F1-9_R, F1-9_E, F1-9_A, F1-9_D (int)
    type(char) : 12 types
'''
sheet1 = icru_2021_data_sheet1.copy()
### Delete '테스트' data
sheet1.drop([0], axis=0, inplace=True)
col = sheet1.columns[1]
for key in ['테스트','d']:
    testRow = sheet1[sheet1[col]==key].index
    sheet1.drop(testRow,inplace=True)

### Column Clean-UP
sheet1.drop(sheet1.columns[-1], axis=1, inplace=True)
sheet1.drop(sheet1.columns[38:50], axis=1, inplace=True)
sheet1.drop(sheet1.columns[1], axis=1, inplace=True)
sheet1 = sheet1[:2160]
orgColumn = list(sheet1.columns)
renamedColumn =['id',
                'F1_R', 'F1_E', 'F1_A', 'F1_D',
                'F2_R', 'F2_E', 'F2_A', 'F2_D',
                'F3_R', 'F3_E', 'F3_A', 'F3_D',
                'F4_R', 'F4_E', 'F4_A', 'F4_D',
                'F5_R', 'F5_E', 'F5_A', 'F5_D',
                'F6_R', 'F6_E', 'F6_A', 'F6_D',
                'F7_R', 'F7_E', 'F7_A', 'F7_D',
                'F8_R', 'F8_E', 'F8_A', 'F8_D',
                'F9_R', 'F9_E', 'F9_A', 'F9_D',
                'type']
sheet1.columns = renamedColumn
columnRef1 = np.array([renamedColumn,orgColumn]).transpose()
sheet1.reset_index(drop=True, inplace=True)

### Row Clean-UP
for col in sheet1.columns[:-1]:
    sheet1[col] = sheet1[col].astype('int64')
sheet1['type'] = sheet1['type'].astype('category')

In [252]:
# move and add 'type' column next to 'id'
sheet0 = pd.concat([sheet0,sheet1['type']], axis=1)
sheet0 = sheet0[['id', 'type', 
                'gender', 'year',
                'work1', 'work2', 'work3',
                'expF1', 'expF2', 'expF3', 'expF4', 'expF5', 'expF6', 'expF7', 'expF8', 'expF9']]
sheet1 = sheet1[['id','type',
                'F1_R', 'F1_E', 'F1_A', 'F1_D',
                'F2_R', 'F2_E', 'F2_A', 'F2_D',
                'F3_R', 'F3_E', 'F3_A', 'F3_D',
                'F4_R', 'F4_E', 'F4_A', 'F4_D',
                'F5_R', 'F5_E', 'F5_A', 'F5_D',
                'F6_R', 'F6_E', 'F6_A', 'F6_D',
                'F7_R', 'F7_E', 'F7_A', 'F7_D',
                'F8_R', 'F8_E', 'F8_A', 'F8_D',
                'F9_R', 'F9_E', 'F9_A', 'F9_D']]

## Analysis

### Case1
Heatmap of READ types in Features (F1~F9)
- 2. READ 분석: 질문별 R,E,A,D 분포를 통해서 질문의 Quality를 파악할 수 있을 거 같음


In [299]:
def featureAnalysis(FeatureName):
    featureCol = ['id','type']

    df_result = pd.DataFrame()
    substring = FeatureName
    for col in sheet1.columns:
        if substring in col:
            featureCol.append(col)
    df = sheet1[featureCol]

    df_typeFiltered = df[df['type'].isin(['RE','RA','RD'])]
    df_result['typeR'] = df_typeFiltered[featureCol[-4:]].mean()

    df_typeFiltered = df[df['type'].isin(['ER','EA','ED'])]
    df_result['typeE'] = df_typeFiltered[featureCol[-4:]].mean()

    df_typeFiltered = df[df['type'].isin(['AR','AE','AD'])]
    df_result['typeA'] = df_typeFiltered[featureCol[-4:]].mean()

    df_typeFiltered = df[df['type'].isin(['DR','DE','DA'])]
    df_result['typeD'] = df_typeFiltered[featureCol[-4:]].mean()

    return df_result

In [315]:
df = featureAnalysis('F1')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F1_R,3.470588,2.713087,2.723232,2.067568
F1_E,3.107843,3.778523,3.007071,3.532432
F1_A,1.960784,1.808725,2.393939,1.875676
F1_D,1.460784,1.699664,1.875758,2.524324


In [316]:
df = featureAnalysis('F2')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F2_R,3.137255,2.469799,2.325253,2.297297
F2_E,1.593137,2.152685,1.555556,1.410811
F2_A,2.720588,2.887584,3.343434,2.970270
F2_D,2.549020,2.489933,2.775758,3.321622


In [317]:
df = featureAnalysis('F3')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F3_R,2.681373,2.318792,2.501010,2.281081
F3_E,3.191176,3.699664,3.021212,3.064865
F3_A,1.955882,1.548658,2.290909,1.321622
F3_D,2.171569,2.432886,2.186869,3.332432


In [318]:
df = featureAnalysis('F4')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F4_R,3.303922,2.395973,2.280808,2.302703
F4_E,2.411765,3.073826,2.291919,2.394595
F4_A,1.745098,1.567114,2.422222,1.591892
F4_D,2.539216,2.963087,3.005051,3.710811


In [319]:
df = featureAnalysis('F5')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F5_R,3.637255,2.538591,2.403030,1.862162
F5_E,2.828431,3.734899,3.093939,3.454054
F5_A,2.250000,2.265101,2.845455,2.267568
F5_D,1.284314,1.461409,1.657576,2.416216


In [320]:
df = featureAnalysis('F6')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F6_R,3.274510,1.991611,1.996970,1.762162
F6_E,2.617647,3.338926,2.465657,2.551351
F6_A,1.848039,1.664430,2.496970,1.983784
F6_D,2.259804,3.005034,3.040404,3.702703


In [321]:
df = featureAnalysis('F6')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F6_R,3.274510,1.991611,1.996970,1.762162
F6_E,2.617647,3.338926,2.465657,2.551351
F6_A,1.848039,1.664430,2.496970,1.983784
F6_D,2.259804,3.005034,3.040404,3.702703


In [322]:
df = featureAnalysis('F7')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F7_R,3.387255,2.117450,2.407071,2.002703
F7_E,2.132353,3.063758,2.029293,2.037838
F7_A,2.441176,2.026846,2.909091,2.351351
F7_D,2.039216,2.791946,2.654545,3.608108


In [323]:
df = featureAnalysis('F8')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F8_R,2.931373,2.209732,1.990909,1.900000
F8_E,3.235294,3.780201,3.269697,3.391892
F8_A,1.578431,1.449664,2.020202,1.610811
F8_D,2.254902,2.560403,2.719192,3.097297


In [324]:
df = featureAnalysis('F9')
df.style.background_gradient(cmap='Greens',vmin = 0, vmax = 4, axis = None)

,typeR,typeE,typeA,typeD
F9_R,2.946078,1.516779,1.847475,1.351351
F9_E,2.681373,3.510067,2.695960,2.756757
F9_A,2.602941,2.713087,3.243434,2.708108
F9_D,1.769608,2.260067,2.213131,3.183784
